In [1]:
import coiled
import dask
import dask.dataframe as dd

In [2]:
cluster = coiled.Cluster(name="read-parquet-demo", n_workers=5)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/rich/live.py:22
1: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Using existing cluster: 'read-parquet-demo'

In [3]:
client = dask.distributed.Client(cluster)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| dask        | 2022.02.0 | 2022.01.1 | 2022.01.1 |
| distributed | 2022.02.0 | 2022.01.1 | 2022.01.1 |
| numpy       | 1.22.2    | 1.21.5    | 1.21.5    |
| pandas      | 1.4.1     | 1.4.0     | 1.4.0     |
+-------------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [7]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    engine="pyarrow",
    storage_options={"anon": True, "use_ssl": True},
)

In [8]:
ddf.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
2000-01-01 00:00:04,967,Michael,-0.251460,0.810930


In [10]:
ddf.dtypes

id        int64
name     object
x       float64
y       float64
dtype: object

## Query without column pruning

In [9]:
%%time

ddf["id"].nunique().compute()

CPU times: user 406 ms, sys: 37.3 ms, total: 443 ms
Wall time: 58.6 s


367

## Query with column pruning

In [11]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    engine="pyarrow",
    storage_options={"anon": True, "use_ssl": True},
    columns=["id"],
)

In [12]:
%%time

ddf["id"].nunique().compute()

CPU times: user 417 ms, sys: 96.6 ms, total: 514 ms
Wall time: 42.6 s


367

## Filter without predicate pushdown

In [13]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    engine="pyarrow",
    storage_options={"anon": True, "use_ssl": True},
)

In [14]:
%%time

len(ddf[ddf.id > 1170])

CPU times: user 502 ms, sys: 48.6 ms, total: 551 ms
Wall time: 1min 17s


65

## Filter with predicate pushdown

In [15]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    engine="pyarrow",
    storage_options={"anon": True, "use_ssl": True},
    filters=[[("id", ">", 1170)]],
)

In [16]:
%%time

len(ddf[ddf.id > 1170])

CPU times: user 33.5 ms, sys: 11.8 ms, total: 45.3 ms
Wall time: 4.51 s


65

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/comm/tcp.py", line 398, in connect
    stream = await self.client.connect(
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/asyncio/task

## ignore _metadata file

In [4]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    engine="pyarrow",
    storage_options={"anon": True, "use_ssl": True},
    ignore_metadata_file=True,
)

In [5]:
ddf.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
2000-01-01 00:00:04,967,Michael,-0.251460,0.810930


## index

In [6]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    engine="pyarrow",
    storage_options={"anon": True, "use_ssl": True},
    index="timestamp",
)

In [7]:
ddf.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
2000-01-01 00:00:04,967,Michael,-0.251460,0.810930


In [8]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    engine="pyarrow",
    storage_options={"anon": True, "use_ssl": True},
    index=False,
)

In [9]:
ddf.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
1,2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2,2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
3,2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
4,2000-01-01 00:00:04,967,Michael,-0.251460,0.810930


## categories

In [11]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    engine="pyarrow",
    storage_options={"anon": True, "use_ssl": True},
    categories=["id"],
)

In [12]:
ddf.dtypes

id      category
name      object
x        float64
y        float64
dtype: object

In [13]:
ddf.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
2000-01-01 00:00:04,967,Michael,-0.251460,0.810930
